**Robotic Mapping with LiDAR Data |** Linear Transformations
============================================================



ROB 101: Computational Linear Algebra | University of Michigan, Department of Robotics



⚠**Warning**⚠: Follow all instructions. We use an auto-grader to check your work. If you invent new notation or variables other than what was listed in the instructions and fail the auto-grader tests, then you will not receive points back. Please be careful.



In [ ]:
using Plots, Random, LinearAlgebra

Task 0
======



This is a warm-up task intended to help you with the later problems. It is highly recommended before completing the graded tasks.



In [ ]:
# Some random data to play with
Random.seed!(101)

A = randn(3,4)
show(stdout, "text/plain", A)
println("\n")

B = randn(4,10)
show(stdout, "text/plain", B)
println("\n")

@show v3 = [1;2;3.0]
@show v4 = [1;2;3;4.0]

**Create variables** `nRows` **and** `nCols` **that store the number of rows and columns in matrix** `A` **using** `size()`**.**



In [ ]:
# YOUR ANSWER HERE
@show nRows == 3
@show nCols == 4

**Create a variable** `nColsB` **that provides the number of columns in the** `B` **matrix using** `size()`**.**



In [ ]:
# YOUR ANSWER HERE
@show nColsB == 10

Here are 3 examples for how to add a size compatible vector (`v3`) to each of the columns of a matrix (`A`).



In [ ]:
#Example 1: Brute Force Method
C = copy(A)

C[:,1] = C[:, 1] + v3
C[:,2] = C[:, 2] + v3
C[:,3] = C[:, 3] + v3
C[:,4] = C[:, 4] + v3

show(stdout, "text/plain", A)
println()
show(stdout, "text/plain", C)
println()
show(stdout, "text/plain", C-A)
println()



In [ ]:
#Example 2: For Loop Method
C = copy(A)

for i = 1:size(C,2)
   #C[:,i] takes all the rows of C and the i-th column
    C[:,i] = C[:,i] + v3 
end

show(stdout, "text/plain", A)
println()
show(stdout, "text/plain", C)
println()
show(stdout, "text/plain", C-A)
println()

In [ ]:
#Example 3: Broadcasting
C = copy(A)

# if you remove the dot before the plus sign, you will generate an error
C = C .+ v3

show(stdout, "text/plain", A)
println()
show(stdout, "text/plain", C)
println()
show(stdout, "text/plain", C-A)
println()

**Use the for loop or broadcasting method to add** `v4` **to each column of matrix** `B`**. Store the result in matrix** `C`**.**



In [ ]:
# YOUR ANSWER HERE
D = copy(B)
@show C[:,1] == D[:, 1] + v4
@show C[:,2] == D[:, 2] + v4
@show C[:,3] == D[:, 3] + v4
@show C[:,4] == D[:, 4] + v4

# This check assumes you put the result in matrix C
E = C - B; test = true
for i = 1:length(v4)
    for j = 1:size(E,2)
        test = test & isapprox(E[i,j], i, atol = 1e-2)
    end
end

if test == true
    println("Good Work")
else
    println("Try again. You will need to store solutions in the appropriate  variables later in this project.")
end

Here are 2 examples of multiplying a size compatible matrix (`A`) into the columns of a second matrix (`B`).



In [ ]:
#Example 1: For Loop Method
nRowsA = size(A,1)
nColsB = size(B,2)

# a matrix of zeros of the correct size to hold the solution
C = zeros(nRowsA, nColsB) 

for k = 1:size(B,2)
    v = B[:,k] # extract the k-th column of B
    Av = A * v # do the multiplication of A times the k-th column of B
    C[:,k] = Av # assign the result to the k-th column of C
end


#Example 1a: a small variation on the above - do it all in one step
D = zeros(nRowsA, nColsB) 

for k = 1:size(B,2)
    D[:,k] = A * B[:,k] 
end

#Example 1b: a small variation on the above - start with a blank matrix

E = Array{Float64}(undef, nRowsA, nColsB)  

for k = 1:size(B,2)
    E[:,k] = A * B[:,k] 
end


#Example 2: Simply use Julia's powerful multiplication!

F = A*B # Yep, it's that simple :)

show(stdout, "text/plain", C)
println()
show(stdout, "text/plain", D)
println()
show(stdout, "text/plain", E)
println()
show(stdout, "text/plain", F)
println()


**Use your favorite method to multiply each column of the matrix** `M` **by the matrix** `N` **and store the result in a matrix called** `myProd`**.**



In [ ]:
Random.seed!(101)
M = randn(11,11)
N = randn(11,11)

In [ ]:
# YOUR ANSWER HERE
@show myProd == M*N

if isapprox(norm(myProd), 43.9059, atol = 1e-2)
    println("Good work. You are ready to take on the Project.")
elseif isapprox(norm(myProd), 35.5951, atol = 1e-2)
    println("You got it backwards - ")
    println("You multiplied each column of M by the matrix N. Try again.")
else
    println("You may want to reach out for help.")
end

Task 1
======



You will plot a simple square and apply different transformations to it. The goal is to learn a matrix version of what mathematicians call "***affine transformations***" and you will learn a special way to write coordinates that robotics engineers use on a regular basis: ***homogeneous coordinates***.



We will tackle translation, rotation, and scaling transformations one-by-one in what follows. Rotation and scaling involve matrix-vector multiplication, while translation involves adding two vectors. Even though addition seems simpler than multiplication, we are going to show you how to turn translation into matrix multiplication! Robotics engineers want to do this so that three operations, translation, rotation, and scaling can be written in exactly the same format. The fruit of our labor will simplify our manipulation of the roughly 185 MB of LiDAR data in Task 3.



In [ ]:
#=
    This function generates a 2D Plot
    Mandatory parameters are
                        xdata -> x coordinates
                        ydata -> y coordinates
                        color -> plot color
=#
function plot_2D(xdata::Vector{Float64}, ydata::Vector{Float64}, color; 
                 new=true, aspect_ratio=:equal, 
                 xlims = (-2,2), ylims = (-2,2), grid=true,
                 framestyle=:origin, legend=false, linewidth=3)
    fxn = new ? plot : plot! 
    fxn(xdata, ydata, seriescolor=color, aspect_ratio=aspect_ratio, xlims=xlims,
        ylims=ylims, grid=grid, framestyle=framestyle, legend=legend, 
        linewidth=linewidth)
end

Read the following code block carefully. You will use the `points` matrix moving forward.



In [ ]:
#=
This cell creates a set of points to form a square when plotted
=#

gr()

#= 
Define points to form a square, note there are five points, because 
we need to come back to the first point to complete the square for 
plotting (connecting lines) 
=#

# The coordinate points are in the form: [x1 y1; x2 y2; ...; x5 y5]' 
points = [0. 0.; 0. 1.;1. 1.; 1. 0.; 0. 0.]';

#= 
After the transpose, the array points looks like this:
[x1 x2 ... x5; 
 y1 y2 ... y5]
=#

# x coordinates extracted from the array
x = points[1,:]

# y coordinates extracted from the array
y = points[2,:]

# plotting the array of points in the same plot as a coordinate frame  
plot_2D(x, y, :green)

### Task 1, Part A: Translation by Addition



**Translate the **x** and **y** values by **-0.5** so the square is centered about the origin.**



\begin{bmatrix} x \\ y \end{bmatrix} \rightarrow \begin{bmatrix} x \\ y \end{bmatrix} + \begin{bmatrix} t\_x \\ t\_y \end{bmatrix}where \begin{bmatrix} t\_x \\ t\_y \end{bmatrix} = \begin{bmatrix} -0.5 \\ -0.5 \end{bmatrix}



The `points` matrix is organized such that the x-values are in the first row and the corresponding y-values are in the second row. The given translation vector, `t`, that hold two values; the [1,1] value is -0.5 which is how much the x-values are translated and the [2,1] value is -0.5 which is how much the y-values are translated by.



In [ ]:
# Define the translation vector t
t = [-0.5; -0.5]

# copy the original points so the original data is not changed
points2 = copy(points)

Modify `points2` so that the x and y values are translated by vector `t`



In [ ]:
# YOUR ANSWER HERE
#= 
if the value of is_it_correct_check1 is "Yes", then your answer 
may be correct. However, if the value of is_it_correct_checkN is "No", 
then your answer is defintely wrong
=#
is_it_correct_check1 = ((points2 - points) == hcat([t for i=1:size(points2,2)]...)) ? "Yes" : "No"

@show is_it_correct_check1;

Run the plot to see your newly translated square. The plot should show the original green square translated to be around the origin and now in the color blue. If that is not the case, then go back and correct your code.



In [ ]:
x2 = points2[1,:];
y2 = points2[2,:];
 
plot_2D(x2, y2, :steelblue, new=false)

### Task 1, Part B: Translation by Matrix Multiplication



**Implement the translation of the square as you did in Part A, but this time using homogeneous coordinates and the given translation matrix (**`T`**).**



In [ ]:
# Make a copy of the original points so they are not overwritten
points3 = copy(points);

To use **homogeneous coordinates** is a simple operation because we just need to append a 1 to each point \begin{bmatrix} x \\ y \end{bmatrix} such that \begin{bmatrix} x \\ y \\ 1 \end{bmatrix}.



Append 1 to each (x,y) coordinate in `points3` to make them homogenous coordinates. Store the result back into `points3`.



In [ ]:
# YOUR ANSWER HERE


**Pause**: Let’s take a look at the math behind the method so you can see for yourself how and why this works.



We claim that we can perform vector translation by multiplying a vector in homogeneous coordinates by the matrix



T = \begin{bmatrix} 1 & 0 & t\_x \\ 0 & 1 & t\_y \\ 0 & 0 & 1 \end{bmatrix}



Let's see if this actually works (*you can confirm the multiplication by hand*):



\begin{bmatrix} 1 & 0 & t\_x \\ 0 & 1 & t\_y \\ 0 & 0 & 1 \end{bmatrix} \* \begin{bmatrix} x \\ y \\ 1 \end{bmatrix} = \begin{bmatrix} x + t\_x \\ y + t\_y \\ 1 \end{bmatrix}



The inverse operation, taking a vector of homogeneous coordinates and dropping the 1, produces the original vector [x;y]. When we drop the 1 to put our transformed vector back in the ordinary Cartesian coordinates we have:



\begin{bmatrix} x + t\_x \\ y + t\_y \\ 1 \end{bmatrix}



**Back to coding**: Apply the translation to each of the points that form our square by multiplying `points3` by the matrix `T` and storing the result back into `points3`.



In [ ]:
# Define the translation matrix T
T = [1 0 -0.5;0 1 -0.5; 0 0 1];

In [ ]:
# YOUR ANSWER HERE


Run the plot to see your newly translated square. The plot should show the original green square translated to be around the origin and now in the color black. If that is not the case, then go back and correct your code.



In [ ]:
x3 = points3[1,:];
y3 = points3[2,:];
 
plot_2D(x3, y3, :black, new=false, linewidth=5)

### Task 1, Part C: Rotation



**Rotate our translated square in the x-y plane by 45 degrees (**\pi / 4**).**



Here is the 2D-rotation matrix in homogenous coordinates:



\begin{bmatrix} x \\ y \\ 1 \end{bmatrix} = \begin{bmatrix} cos(\theta) & -sin(\theta) & 0 \\ sin(\theta) & cos(\theta) & 0 \\ 0 & 0 & 1 \end{bmatrix} \cdot \begin{bmatrix} x \\ y \\ 1 \end{bmatrix}



Apply the rotation above by using the given transformation matrix above (name it `T`). Yes, we want to use `points3` again because we are applying the rotation to our already translated matrix.



⚠**Warnings**:



* Do not create a new `points3` vector and do not reinitialize a new copy of `points`.
* Your answer for Part B must be correct in order for this Part C to be correct.
* The auto-grader assumes your transformation matrix is called `T`.
* Do not leave out the sin and cos terms out of your transformation matrix.


In [ ]:
# YOUR ANSWER HERE
is_it_correct_check1 = abs(T[1,1]-cos(pi/4)) < 1e-4 ? "Yes" : "No"
is_it_correct_check2 = abs(1-det(T[1:2,1:2])) < 1e-4 ? "Yes" : "No"
is_it_correct_check3 = abs(points3[1,2]+0.707107 ) < 1e-4 ? "Yes" : "No"

@show is_it_correct_check1;  # Transformation
@show is_it_correct_check2;  # Transformation
if (is_it_correct_check1 == "No")||(is_it_correct_check2 == "No")
    println("You failed at least one test on the transformation matrix, T. Two common mistakes are:")
    println("   a) You did not call your transformation T")
    println("   b) You failed to use the appropriate trig functions in your transformation")
end
@show is_it_correct_check3; # Points vector
if is_it_correct_check3 == "No"
    println("If you failed the transformation check, then that is probably why you failed this test.")
    println("If you passed the transformation check, then did you use the correct points vector. Double check, please!? ")
    println("If you passed the transformation check, and you used the correct points vector, 
        then did you place the transformed points back into points3? ")
end

Run the plot to see the newly rotated square, which is a diamond. The plot should show the original green square translated to be around the origin, rotated to form a diamond, and now in the color red. If that is not the case, then go back and correct your code.



In [ ]:
x3 = points3[1,:]
y3 = points3[2,:]

plot_2D(x3, y3, :red, new=false, linewidth=5)

### Task 1, Part D: Scaling



**Scale the rotated square.**



Here is the 2D-scale transformation matrix in homogenous coordinates:



\begin{bmatrix} x \\ y \\ 1 \end{bmatrix} = \begin{bmatrix} S\_x & 0 & 0 \\ 0 & S\_y & 0 \\ 0 & 0 & 1 \end{bmatrix} \cdot \begin{bmatrix} x \\ y \\ 1 \end{bmatrix}



Apply the scale above by using the given transformation matrix above (name it `T`). S\_x = 1.5 and S\_y = 0.5 which means our diamond will be stretched in the x direction and compressed in the y direction. Yes, we want to use `points3` again because we are applying the scaling to our already translated and rotated matrix.



In [ ]:
# YOUR ANSWER HERE


Run the plot to see the newly scaled diamond. The plot should show the original green square translated to be around the origin, rotated to form a diamond, scaled to be wide and short, and now in the color purple. If that is not the case, then go back and correct your code.



In [ ]:
x3 = points3[1,:]
y3 = points3[2,:]

plot_2D(x3, y3, :purple, new=false, linewidth=5)

### Task 1, Part E: Affine Transformations



#### **Implement a sequence of transformations on the original square.**



All the transformations we have done so far (translation, rotation, and scaling) and combinations of them are called **Affine Transformations**. When we work with big data sets and we need to apply a sequence of translations, rotations, and scalings to a set of points, you can imagine that keeping track of everything during the transformation processes could become cumbersome over time. The saving feature of using the abstract notion of an Affine Transformation is that we can build a complicated transformation by multiplying together the individual transformations considered one at a time. For example, let's suppose we want to translate a set of points by [-0.5;-0.5], then rotate the translated object counterclockwise by 45 degrees, and then finally scale the translated and rotated object! That's a lot of transformations as you know… you just did it!



You’ll be delighted to find out that we can write the overall transformation as simply as this:



T = \begin{bmatrix} 1.5 & 0 & 0 \\ 0 & 0.5 & 0 \\ 0 & 0 & 1 \end{bmatrix} \cdot \begin{bmatrix} cos(\pi/4) & -sin(\pi/4) & 0 \\ sin(\pi/4) & cos(\pi/4) & 0 \\ 0 & 0 & 1 \end{bmatrix} \cdot \begin{bmatrix} 1 & 0 & -0.5 \\ 0 & 1 & -0.5 \\ 0 & 0 & 1 \end{bmatrix}



We prepared the overall homogeneous transformation by multiplying three individual homogeneous transformations, which we can then apply all at once to our point \begin{bmatrix} x \\ y \end{bmatrix}, once we express it in homogeneous coordinates as \begin{bmatrix} x \\ y \\ 1 \end{bmatrix}.



Build the transformation matrix `T` and then apply it to our vector `points4`. Whatever you do, do not multiply out the above matrices by hand and then enter one matrix. Write `T` as the product of three homogeneous matrices and let Julia do the matrix multiplication for you.



In [ ]:
#=
A copy of the original points
with 1 appended to each point 
=#
points4 = [copy(points); ones(1,5)]

In [ ]:
# YOUR ANSWER HERE


Check your work via this plot. If it's not the same purple diamond you saw before, go back and double check your work.



In [ ]:
x4 = points4[1,:]
y4 = points4[2,:]

plot_2D(x, y, :green)
plot_2D(x4, y4, :purple, new=false, linewidth=5)

Task 2
======



This task is worth 2 points.



In this task, you are provided a distorted image which makes the text hard to read. You will apply an affine transformation to correct the point cloud and make the text readable.



A **point cloud** is simply a collection of points in space (3D and more) or a plane (2D). For a 2D point cloud, the first two rows of the data contain the x and y coordinates of our point, respectively. The third row contains additional information about our point, which for the case of a LiDAR sensor is the Intensity, and for a camera is the RGB values (red, green, blue color information).



For your 2D point cloud, think of it as a camera image. The point cloud is thus in a plane and the color data is given in simplified RGB encoding with values between (0, 1). Think about it as a rainbow, where red has value 0 and blue has value 1.



In [ ]:
using DelimitedFiles, Plots

for (root, dirs, files) in walkdir("/home/jovyan/")
    println("Root: ", root)
    println("Directories: ", dirs)
    println("Files: ", files)
end

In [ ]:

#=
    This function generates a 2D Point Cloud Plot
    Mandatory parameters are
                        xdata -> x coordinates
                        ydata -> y coordinates
                        color -> pointcloud color
=#
function plot_2D_pointcloud(xdata::Vector{Float64}, ydata::Vector{Float64}, color; 
                new=true, seriestype=:scatter, markersize = 0.8, 
                markerstrokewidth = 0, dpi = 150, legend = false,
                markercolor  = cgrad(:rainbow, rev = true))
    fxn = new ? plot : plot!
    fxn(xdata, ydata, seriestype=seriestype, markersize=markersize, 
        markerstrokewidth=markerstrokewidth, dpi=dpi, 
        markercolor=markercolor, marker_z=color, legend=legend)
end


First, we read the contents of the file "question\_image.csv" into an array (`pointcloud`) and plot the points in a scatter plot.



In [ ]:
default(fmt = :png)

pointcloud = readdlm("question_image_1.csv",',')

# The x and y coordinates for the first 2 rows of the data 
# can be extracted from the array separately
point_data = pointcloud[1:2,:]

# The color data of the pointcloud forms the last row, and 
# can be saved separately
println("Look at the size of the data you are manipulating! Yes, this is more interesting than the square.")
println(" ")
println("The vector pointcloud[3,:] has $(length(pointcloud[3,:])) elements. Oh my gosh!")
color_data = pointcloud[3,:]

In [ ]:
# Plot the 2D point cloud data with color values! 
plot_2D_pointcloud(point_data[1,:], point_data[2,:], color_data)

The homogeneous transformation for correcting the image above is known:



T = \begin{bmatrix} -0.09239 & 0.038268 & 300 \\ -0.38268 & -0.923879 & 165 \\ 0 & 0 & 1 \end{bmatrix}



**Apply** `T` **to the distorted point cloud so the corrected image can be plotted.**



1. Build the transformation matrix and store it in variable `T` (be careful to copy the numbers correctly!)
2. Apply the transformation, `T`, to all of the points, `X`
3. Store your results back into `X`


In [ ]:
# Homogenous points, X, created for you
X = [copy(point_data); ones(1,size(point_data,2))]

In [ ]:
# YOUR ANSWER HERE
is_it_correct_checkTa = isapprox(sum(T) - sum(abs.(T)), -2.797897999999975, atol = 1e-6 )
is_it_correct_checkTb = isapprox(sum(T) + sum(abs.(T)), 932.076536, atol = 1e-6 ) 

if is_it_correct_checkTa & is_it_correct_checkTb
    println("The transformation T is likely correct.")
    println(" ")
else
    println("At least one entry of the transformation matrix T is incorrect.")

    println(" ")
end

# Now we check your data in X. Errors in X could arise from T being incorrect
# or from an incorrect application of the transformation T to the data in X
# or from you storing your data in a variable other than X, which will cause lots of problems

is_it_correct_check1 = abs((X[3, :] - ones(size(X, 2)))[1]) < 1e-4 ? "Yes" : "No"
is_it_correct_check2 = abs(sum(X[2, :]) - -54512.825647621416) < 1e-4 ? "Yes" : "No"
is_it_correct_check3 = abs(sum(X[:,1]) - 164.00782569999998) < 1e-4 ? "Yes" : "No"
is_it_correct_check4 = abs(sum(X[:, 5]) - 152.0378589000001) < 1e-4 ? "Yes" : "No"
is_it_correct_check5 = abs(sum(X) - -181258.1626895815)< 1e-4 ? "Yes" : "No"


@show is_it_correct_check1;
@show is_it_correct_check2;
@show is_it_correct_check3;
@show is_it_correct_check4;
@show is_it_correct_check5;is_it_correct_checkTa = isapprox(sum(T) - sum(abs.(T)), -2.797897999999975, atol = 1e-6 )


Check your work via this plot. You should be able to read the text clearly.



In [ ]:
plot_2D_pointcloud(X[1,:], X[2,:], color_data)

Submit & Go to Task 3!
======================

